Lab: 最大概似估計
================

In [1]:
import torch

In [2]:
from torch.distributions import Normal
normal = Normal(loc=0., scale=1.)

In [3]:
print("random sample with shape ():\n",
      normal.sample())
print("random sample with shape (3,):\n",
      normal.sample(sample_shape=(3,)))
print("random sample with shape (2,3):\n",
      normal.sample(sample_shape=(2, 3)))

random sample with shape ():
 tensor(-1.7886)
random sample with shape (3,):
 tensor([-1.2067, -1.5614,  1.1364])
random sample with shape (2,3):
 tensor([[ 1.0188,  1.3026,  0.4082],
        [ 0.4221, -0.8592,  0.0048]])


In [4]:
print("cumulative probability given value with shape ():\n",
      normal.cdf(value=0), "\n")
print("cumulative probability given value with (3,):\n",
      normal.cdf(value=torch.Tensor([-1, 0, .5])), "\n")
print("cumulative probability given value with (2,3):\n",
      normal.cdf(value=torch.Tensor([[-1, 0, .5], [-2, 1, 3]])))

cumulative probability given value with shape ():
 tensor(0.5000) 

cumulative probability given value with (3,):
 tensor([0.1587, 0.5000, 0.6915]) 

cumulative probability given value with (2,3):
 tensor([[0.1587, 0.5000, 0.6915],
        [0.0228, 0.8413, 0.9987]])


In [5]:
print("cumulative probability given value with shape ():\n",
      normal.log_prob(value=0), "\n")
print("cumulative probability given value with (3,):\n",
      normal.log_prob(value=torch.Tensor([-1, 0, .5])), "\n")
print("cumulative probability given value with (2,3):\n",
      normal.log_prob(value=torch.Tensor([[-1, 0, .5], [-2, 1, 3]])))

cumulative probability given value with shape ():
 tensor(-0.9189) 

cumulative probability given value with (3,):
 tensor([-1.4189, -0.9189, -1.0439]) 

cumulative probability given value with (2,3):
 tensor([[-1.4189, -0.9189, -1.0439],
        [-2.9189, -1.4189, -5.4189]])


In [6]:
print(normal)

Normal(loc: 0.0, scale: 1.0)


In [7]:
print(normal.batch_shape)
print(normal.event_shape)

torch.Size([])
torch.Size([])


In [8]:
normal_batch = Normal(loc=torch.Tensor([0., 1.]), scale=torch.Tensor([1., 1.5]))
print(normal_batch)

Normal(loc: torch.Size([2]), scale: torch.Size([2]))


In [9]:
print("random sample with sample_shape ():\n",
      normal_batch.sample(), "\n")
print("random sample with sample_shape (3,):\n",
      normal_batch.sample(sample_shape=(3,)), "\n")
print("random sample with sample_shape (2,3):\n",
      normal_batch.sample(sample_shape=(2,3)))


random sample with sample_shape ():
 tensor([-0.6340,  3.3679]) 

random sample with sample_shape (3,):
 tensor([[-0.4915,  1.5562],
        [ 0.5452,  2.3304],
        [-0.0674,  1.6319]]) 

random sample with sample_shape (2,3):
 tensor([[[-1.0306,  2.5778],
         [ 0.0607,  1.7681],
         [ 0.4815, -0.9345]],

        [[ 0.5089,  2.7806],
         [-0.5045,  1.4897],
         [ 0.8944,  0.1244]]])


In [10]:
print("log-probability given value with shape ():\n",
      normal_batch.log_prob(0), "\n")
print("log-probability given value with shape (2,):\n",
      normal_batch.log_prob(torch.Tensor([0, 0])), "\n")
print("log-probability given value with shape (2,1):\n",
      normal_batch.log_prob(torch.Tensor([[0], [0]])))

log-probability given value with shape ():
 tensor([-0.9189, -1.5466]) 

log-probability given value with shape (2,):
 tensor([-0.9189, -1.5466]) 

log-probability given value with shape (2,1):
 tensor([[-0.9189, -1.5466],
        [-0.9189, -1.5466]])


In [11]:
from torch.distributions import MultivariateNormal
mvn = MultivariateNormal(
    loc=torch.Tensor([0, 1]),
    scale_tril=torch.cholesky(torch.Tensor([[1., 0.], [0., .5]])))
print(mvn)


MultivariateNormal(loc: torch.Size([2]), scale_tril: torch.Size([2, 2]))


In [12]:
print("random sample with sample_shape ():\n",
      mvn.sample(), "\n")
print("random sample with sample_shape (3,):\n",
      mvn.sample(sample_shape=(3,)), "\n")
print("random sample with sample_shape (2, 3):\n",
      mvn.sample(sample_shape=(2, 3)))

random sample with sample_shape ():
 tensor([-0.8543,  1.4047]) 

random sample with sample_shape (3,):
 tensor([[-0.0202,  0.3989],
        [-0.5020,  0.0073],
        [ 0.4557,  0.4627]]) 

random sample with sample_shape (2, 3):
 tensor([[[-0.9723,  0.4815],
         [-1.3788,  0.1017],
         [ 0.0940,  0.0351]],

        [[-1.1022,  1.3271],
         [-0.1163,  1.0023],
         [ 1.1889,  1.5059]]])


In [13]:
print("log-probability given value with shape (2,):\n",
      mvn.log_prob(torch.Tensor([0, 0])), "\n")
print("log-probability given value with shape (2,1):\n",
      mvn.log_prob(torch.Tensor([[0, 0], [0, 0]])))

log-probability given value with shape (2,):
 tensor(-2.4913) 

log-probability given value with shape (2,1):
 tensor([-2.4913, -2.4913])


In [14]:
from torch.distributions import Independent
normal_batch = Independent(normal_batch, reinterpreted_batch_ndims=1)
print(normal_batch.batch_shape)
print(normal_batch.event_shape)


torch.Size([])
torch.Size([2])


In [15]:
mvn_batch = MultivariateNormal(
    loc=torch.Tensor([[0, 1],[1, 2],[2, 3]]),
    scale_tril=torch.cholesky(torch.Tensor([[1., .2], [.2, .5]])))
mvn_batch

MultivariateNormal(loc: torch.Size([3, 2]), scale_tril: torch.Size([3, 2, 2]))

In [16]:
print("random sample with sample_shape ():\n",
      mvn_batch.sample(), "\n")
print("random sample with sample_shape (3,):\n",
      mvn_batch.sample(sample_shape=(3,)), "\n")
print("random sample with sample_shape (2, 3):\n",
      mvn_batch.sample(sample_shape=(2, 3)))


random sample with sample_shape ():
 tensor([[-0.6048,  0.4761],
        [ 1.2854,  1.3377],
        [ 1.1139,  2.9619]]) 

random sample with sample_shape (3,):
 tensor([[[ 1.2273,  1.3594],
         [-0.9842,  0.5286],
         [ 2.5206,  1.9293]],

        [[-0.2119,  1.0673],
         [ 0.3231,  0.9239],
         [ 2.5649,  2.4712]],

        [[-0.0410,  2.0827],
         [ 0.6460,  2.1593],
         [ 1.8229,  3.0561]]]) 

random sample with sample_shape (2, 3):
 tensor([[[[-0.7271,  1.0198],
          [ 2.3459,  1.4973],
          [ 3.4194,  2.5100]],

         [[-0.5421,  0.9143],
          [ 1.3577,  1.9049],
          [ 0.3631,  2.2086]],

         [[-0.6288,  0.8936],
          [ 0.7414,  1.9123],
          [ 2.6031,  2.8700]]],


        [[[-0.2162,  1.9304],
          [ 1.1326,  2.1860],
          [ 1.1356,  2.3910]],

         [[ 0.3058,  0.9221],
          [ 0.5956,  1.9632],
          [ 2.7523,  2.8059]],

         [[-1.2248, -0.3657],
          [ 0.4758,  1.2733],
     

In [17]:
mu_true = torch.tensor([5.])
sigma_true = torch.tensor([2.])
normal_model_true = Normal(
    loc=mu_true,
    scale=sigma_true)
print("normal model:\n", normal_model_true, "\n")

normal model:
 Normal(loc: tensor([5.]), scale: tensor([2.])) 



In [18]:
sample_size = 1000
x = normal_model_true.sample(sample_shape=(sample_size,))
loss_value = -torch.mean(torch.sum(normal_model_true.log_prob(x), dim = 1))
print("negative likelihood value is", loss_value)


negative likelihood value is tensor(2.1340)


In [19]:
epochs = 200
lr = 1.0
mu = torch.tensor([0.], requires_grad=True)
sigma = torch.tensor([1.], requires_grad=True)
optimizer = torch.optim.Adam([mu, sigma], lr=.5)
for epoch in range(epochs):
    normal_model = Normal(loc=mu, scale=sigma)
    loss_value = -torch.mean(normal_model.log_prob(x))
    optimizer.zero_grad()
    loss_value.backward() # compute the gradient
    optimizer.step()

In [20]:
print("ML mean by gradient descent:", mu)
print("ML std by gradient descent:", sigma)

ML mean by gradient descent: tensor([5.0764], requires_grad=True)
ML std by gradient descent: tensor([2.0438], requires_grad=True)


In [21]:
print("ML mean by formula:", torch.mean(x))
print("ML std by formula:", torch.std(x, unbiased=False))

ML mean by formula: tensor(5.0764)
ML std by formula: tensor(2.0420)


In [22]:
mu_true = torch.tensor([-1., 0., 1.])
sigma_tril_true = torch.tensor([[3., 0., 0.], [2., 1., 0.], [.4, .5, .5]])
mvn_model_true = MultivariateNormal(
    loc=mu_true,
    scale_tril=sigma_tril_true)
print("true mean vector: \n", mvn_model_true.mean)
print("true covariance matrix: \n", mvn_model_true.covariance_matrix)

true mean vector: 
 tensor([-1.,  0.,  1.])
true covariance matrix: 
 tensor([[9.0000, 6.0000, 1.2000],
        [6.0000, 5.0000, 1.3000],
        [1.2000, 1.3000, 0.6600]])


In [23]:
sample_size = 1000
x = mvn_model_true.sample(sample_shape=(sample_size,))
loss_value = -torch.mean(mvn_model_true.log_prob(x))
print("negative likelihood value is", loss_value)


negative likelihood value is tensor(4.6755)


In [24]:
epochs = 500
lr = .1
mu = torch.tensor(
    [0., 0., 0.], requires_grad=True)
sigma_tril = torch.tensor(
    [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]],
    requires_grad=True)
optimizer = torch.optim.Adam([mu, sigma_tril], lr=lr)

for epoch in range(epochs):
    mvn_model = MultivariateNormal(
    loc=mu,
    scale_tril=sigma_tril)
    loss_value = -torch.mean(mvn_model.log_prob(x))
    optimizer.zero_grad()
    loss_value.backward() # compute the gradient
    optimizer.step()

In [25]:
print("ML mean by gradient descent: \n",
      mu)
print("ML covariance by gradient descent: \n",
      sigma_tril @ torch.transpose(sigma_tril, 0, 1))

ML mean by gradient descent: 
 tensor([-0.8203,  0.1242,  1.0335], requires_grad=True)
ML covariance by gradient descent: 
 tensor([[8.9358, 5.9664, 1.2385],
        [5.9664, 4.9729, 1.3016],
        [1.2385, 1.3016, 0.6590]], grad_fn=<MmBackward>)


In [26]:
sample_mean = torch.mean(x, dim = 0)
sample_moment2 = (torch.transpose(x, 0, 1) @ x) / sample_size
sample_cov = sample_moment2 - torch.ger(sample_mean, sample_mean)
print("ML mean by formula: \n",
      sample_mean)
print("ML covariance by formula: \n",
      sample_cov)

ML mean by formula: 
 tensor([-0.8203,  0.1242,  1.0335])
ML covariance by formula: 
 tensor([[8.9358, 5.9664, 1.2385],
        [5.9664, 4.9729, 1.3016],
        [1.2385, 1.3016, 0.6590]])
